In [1]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from util_checklist import sample_testcases, get_ingredient_set
import json
# https://github.com/marcotcr/checklist/blob/master/notebooks/tutorials/1.%20Generating%20data.ipynb 

In [2]:
ingredients_tups = get_ingredient_set('recipe_ingredient_dataset/train.json', 'recipe_ingredient_dataset/test.json')

49718 4971.8


In [9]:
def map_more_ingredients():
    types = {}
    types['boneless skinless chicken breasts'] = ['chicken breasts', 'boneless chicken', 'skinless chicken']
    types['red pepper flakes'] = ['red pepper', 'red pepper flakes']
    types['fresh basil leaves'] = ['fresh basil leaves', 'fresh basil']
    types['shredded mozzarella cheese'] = ['shredded mozzarella cheese', 'shredded cheese', 'shredded mozzarella']
    types['fat free less sodium chicken broth'] = []
    types['low sodium soy sauce'] = ['low soy sauce', 'sodium soy sauce']
    types['flat leaf parsley'] = ['leaf parsley', 'flat leaf parsley', 'flat parsley']
    types['chopped cilantro fresh'] = ['chopped cilantro fresh', 'chopped cilantro', 'fresh cilantro']
    types['boneless skinless chicken breast halves'] = []
    return types

In [3]:
editor = Editor()

In [4]:
print(list(editor.lexicons.keys()))

['sentiment', 'male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from']


In [14]:
ing_1_sentences = ['I would like a dish with {ing}', 
                   'Show me {ing} recipes', 
                   'Can I see recipes with {ing}',
                   'I want {ing} in my recipe']
# TODO: come up with a way to generate ing indices.
se_tups = {}
for i in range(len(ing_1_sentences)):
    sentence = ing_1_sentences[i]
    start_indx = sentence.index('{ing}')
    end_indx = start_indx + 5
    end_indx_frm_end = len(sentence) - end_indx
    se_tups[i] = (start_indx, end_indx_frm_end)

def add_type_info(data):
    return [str(i) + '\t' + data[i] for i in range(len(data))]

In [12]:
ingredients = [x[0] for x in ingredients_tups]
ingredients_updates = []
ing_maps = map_more_ingredients()
for ingredient in ingredients:
    if ingredient in ing_maps:
        ingredients_updates.extend(ing_maps[ingredient])
    else:
        ingredients_updates.append(ingredient)
len(ingredients_updates)

208

In [18]:
# single ingredient
ret = editor.template(add_type_info(ing_1_sentences), 
                      ing=ingredients_updates, 
                      unroll=True,
                      remove_duplicates=True)

In [19]:
small_dataset = sample_testcases(ret.data, 500)

In [20]:
small_dataset

array(['2\tCan I see recipes with worcestershire sauce',
       '2\tCan I see recipes with peanuts',
       '0\tI would like a dish with peanut oil',
       '2\tCan I see recipes with vegetable oil',
       '1\tShow me spring onions recipes', '3\tI want cumin in my recipe',
       '0\tI would like a dish with capers',
       '3\tI want ground pepper in my recipe',
       '3\tI want green onions in my recipe',
       '2\tCan I see recipes with flat parsley',
       '0\tI would like a dish with fresh parsley',
       '2\tCan I see recipes with cooking spray',
       '0\tI would like a dish with leaf parsley',
       '3\tI want all-purpose flour in my recipe',
       '3\tI want ground pepper in my recipe',
       '0\tI would like a dish with shredded cheddar cheese',
       '1\tShow me shredded cheddar cheese recipes',
       '1\tShow me purple onion recipes',
       '2\tCan I see recipes with bacon',
       '0\tI would like a dish with fresh basil leaves',
       '3\tI want shallots in m

In [21]:
def convert_checklist_ds(ck_dataset, se_tups):
    label_name = "INGREDIENT"
    converted_ds = []
    for data in ck_dataset:
        sen_type, text = data.split('\t')
        start,end = se_tups[int(sen_type)]
        label_start = start
        label_end = len(text) - end
        annotated = [text]
        annotated.append({'entities': [[label_start, label_end, label_name]]})
        converted_ds.append(annotated)
    return converted_ds

In [22]:
converted_ds_list = convert_checklist_ds(small_dataset, se_tups)

In [23]:
with open('checklist.json', 'w') as fobj:
    json.dump(converted_ds_list, fobj)